In [2]:
import pandas as pd
import nltk
import spacy
import gensim
import seaborn as sb
from gensim import corpora, models, similarities
from spacy.lang.en import English
from nltk.corpus import wordnet as wn

In [44]:
data=pd.read_csv("GuilfordCounty_original_data.csv")

In [33]:
data.head()

,0,1,2
0,NaN,page_number,word
1,1.0,2,guilford
2,2.0,2,county
3,3.0,2,by
4,4.0,2,the


In [34]:
data.describe()

,0
count,94559.000000
mean,47280.000000
std,27296.976389
min,1.000000
25%,23640.500000
50%,47280.000000
75%,70919.500000
max,94559.000000


In [35]:
#filtering stopwords
nltk.download('stopwords')
stop=set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\messi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:
#cleaning 
spacy.load('en')
parser=English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.orth_.startswith('and'):
            lda_tokens.append('the')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [37]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\messi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [38]:
def get_lemma(word):
    lemma=wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

In [39]:
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [40]:
#preparing for lda
def prepare_lda(text):
    tokens=tokenize(text)
    tokens=[token for token in tokens if len(token)>4]
    tokens=[token for token in tokens if token not in stop]
    tokens=[get_lemma(token) for token in tokens]
    return tokens

In [41]:
#creating tokens
import random
text_data=[]
with open('GuilfordCounty_original_data.csv') as f:
    for line in f:
        tokens=prepare_lda(line)
        if random.random()>.99:
            print(tokens)
            text_data.append(tokens)

['118",2,"open']
['122",2,"at']
['218",3,"q']
['291",3,"contest']
['464",5,"a']
['479",5,"believe']
['523",7,"xv']
['533",7,"process']
['603",7,"noneducation']
['629",7,"guilford']
['810",9,"planning']
['867",11,"may']
['909",11,"carlvena']
['919",11,"guilford']
['939",11,"budget']
['1050",11,"the']
['1070",11,"county']
['1128",11,"has']
['1358",12,"revenue']
['1396",12,"does']
['1475",12,"countys']
['1693",13,"general']
['1812",13,"by']
['1954",14,"this']
['2081",15,"revenues']
['2142",15,"these']
['2231",15,"revenues']
['2238",15,"fiscal']
['2249",15,"january']
['2389",15,"m']
['2409",15,"over']
['2442",15,"increase']
['2505",15,"both']
['2597",16,"of']
['2611",16,"elections']
['2702",16,"those']
['2717",16,"revenues']
['2747",16,"emergency']
['2836",16,"by']
['2864",16,"balance']
['2874",16,"all']
['2968",17,"ten']
['3006",17,"of']
['3106",17,"government']
['3475",18,"the']
['3693",18,"next']
['3852",19,"increase']
['3899",19,"mostly']
['3918",19,"being']
['4030",19,"services']
['41

['32013",149,"access']
['32172",150,"afghanistan']
['32360",151,"of']
['32481",151,"examiner']
['32738",152,"remaining']
['32829",153,"public']
['32870",153,"shelter']
['32913",153,"general']
['32963",153,"cruelty']
['32992",154,"services']
['33002",154,"with']
['33031",154,"at']
['33110",154,"are']
['33264",154,"assumed']
['33312",154,"shelter']
['33687",156,"but']
['33702",156,"physical']
['33837",157,"recomm']
['33992",158,"behavior']
['34163",158,"fy']
['34287",158,"daily']
['34309",158,"outofcounty']
['34373",159,"and']
['34440",159,"workshops']
['34443",159,"our']
['34480",159,"with']
['34608",160,"will']
['34616",160,"keep']
['34617",160,"older']
['34660",161,"amended']
['34662",161,"adopted']
['34678",161,"garage']
['34836",162,"strive']
['34882",162,"the']
['34965",162,"on']
['34977",162,"nc']
['35157",163,"other']
['35343",163,"significant']
['35409",163,"coordinated']
['35517",164,"volume']
['35652",164,"to']
['35658",164,"behalf']
['35737",164,"to']
['35797",164,"comprehens

['68214",307,"fire']
['68414",308,"fire']
['68436",309,"retain']
['68454",309,"fy']
['68508",309,"programs']
['68512",309,"and']
['68613",311,"wellness']
['68749",311,"payments']
['68804",312,"and']
['68845",312,"covered']
['68885",312,"medicare']
['69087",314,"health']
['69205",314,"million']
['69304",314,"the']
['69523",316,"safety']
['69659",316,"in']
['69709",316,"to']
['69887",317,"of']
['69948",317,"the']
['69961",317,"shift']
['70071",319,"may']
['70189",319,"equipment']
['70306",319,"are']
['70655",322,"kitchen']
['70668",322,"golf']
['70890",323,"lighting']
['70892",323,"lighting']
['70936",324,"caulking']
['71242",326,"hvac']
['71403",327,"security']
['71449",327,"major']
['71491",327,"exterior']
['71583",328,"base']
['71650",328,"exterior']
['71676",329,"category']
['71998",331,"accessibility']
['72140",331,"parking']
['72225",332,"other']
['72229",332,"interior']
['72246",332,"enforcement']
['72415",333,"accessibility']
['72697",335,"public']
['73019",337,"park']
['73120",3

In [42]:
#creating dictionary
from gensim import corpora
dictionary=corpora.Dictionary(text_data)
corpus= [dictionary.doc2bow(text) for text in text_data]
import pickle 
pickle.dump(corpus, open('corpus.pkl', 'wb'))

In [43]:
#example
ldamodel=gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=6)
for idx, topic in ldamodel.print_topics(-1):
    print('Topic: {} word: {}'.format(idx, topic))

Topic: 0 word: 0.003*"76076",354,"and" + 0.003*"13243",61,"wellness" + 0.003*"68749",311,"payments" + 0.003*"69887",317,"of" + 0.003*"50303",225,"new" + 0.003*"40272",183,"building" + 0.003*"5580",23,"services" + 0.003*"47766",216,"through" + 0.003*"45801",208,"burmil" + 0.003*"21268",99,"retail"
Topic: 1 word: 0.003*"8904",36,"regular" + 0.003*"54431",245,"reliance" + 0.003*"10104",40,"of" + 0.003*"58888",261,"move" + 0.003*"79020",372,"amenities" + 0.003*"74556",345,"over" + 0.003*"73213",339,"license" + 0.003*"85181",402,"micro" + 0.003*"78268",368,"total" + 0.003*"10904",43,"clients"
Topic: 2 word: 0.003*"33002",154,"with" + 0.003*"93830",489,"public" + 0.003*"78740",371,"and" + 0.003*"24815",116,"the" + 0.003*"59698",264,"goals" + 0.003*"49833",223,"employees" + 0.003*"92886",482,"park" + 0.003*"27590",129,"managed" + 0.003*"65873",291,"other" + 0.003*"80625",383,"area"
Topic: 3 word: 0.003*"7551",32,"projects" + 0.003*"77480",362,"law" + 0.003*"45214",206,"northeast" + 0.003*"789